In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import numpy as np

In [2]:
# Device setup (switch to "cpu" if there's a memory issue)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [3]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
# Load dataset
dataset = datasets.ImageFolder('C:\DatasetMajor\PlantVillage', transform=transform)

In [5]:
# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [6]:
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)

In [7]:
# Model setup
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(dataset.classes))  # Update final layer for custom classes
model = model.to(device)

C:\Users\hdnis\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hdnis\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# Training loop
num_epochs = 10
print("Starting training...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    print(f"Epoch {epoch + 1}/{num_epochs}")
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # Print progress within each batch
        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch + 1} completed, Loss: {epoch_loss:.4f}")

Starting training...
Epoch 1/10
Batch 0/16, Loss: 1.0963
Batch 10/16, Loss: 0.3191
Epoch 1 completed, Loss: 0.3014
Epoch 2/10
Batch 0/16, Loss: 0.0712
Batch 10/16, Loss: 0.1851
Epoch 2 completed, Loss: 0.2277
Epoch 3/10
Batch 0/16, Loss: 0.0023
Batch 10/16, Loss: 0.1919
Epoch 3 completed, Loss: 0.3174
Epoch 4/10
Batch 0/16, Loss: 0.2413
Batch 10/16, Loss: 0.0266
Epoch 4 completed, Loss: 0.1766
Epoch 5/10
Batch 0/16, Loss: 0.3771
Batch 10/16, Loss: 0.2257
Epoch 5 completed, Loss: 0.2457
Epoch 6/10
Batch 0/16, Loss: 0.0114
Batch 10/16, Loss: 0.0502
Epoch 6 completed, Loss: 0.1062
Epoch 7/10
Batch 0/16, Loss: 0.0091
Batch 10/16, Loss: 0.1369
Epoch 7 completed, Loss: 0.0433
Epoch 8/10
Batch 0/16, Loss: 0.0080
Batch 10/16, Loss: 0.0172
Epoch 8 completed, Loss: 0.0627
Epoch 9/10
Batch 0/16, Loss: 0.0254
Batch 10/16, Loss: 0.1839
Epoch 9 completed, Loss: 0.0593
Epoch 10/10
Batch 0/16, Loss: 0.0086
Batch 10/16, Loss: 0.0513
Epoch 10 completed, Loss: 0.0718


In [10]:
from PIL import Image  # Import the Image class from PIL

def predict_image(image_path):
    image = Image.open(image_path)  # Open the image
    image = transform(image).unsqueeze(0).to(device)  # Apply transformations
    model.eval()
    with torch.no_grad():
        output = model(image)  # Get the model output
        _, predicted = torch.max(output, 1)  # Get the predicted class
    return dataset.classes[predicted.item()]  # Return the predicted class label

# Example usage with proper file path formatting
print(predict_image(r'C:\DatasetMajor\Test\Blackspot.jpeg'))  # Using a raw string for the file path

Tomato_Bacterial_spot
